# <b>Segmenting and Clustering Neighborhoods in Toronto</b>

## In this project we will explore, segment and cluster the neighborhoods in the city of Toronto

### Part 1: Scraping neighborhood data for Toronto from the web.

In the first part, we need to obtain the Neighborhood data for Toronto. This data is not readily available on the Internet. So we will scrape this data from the following Wikipedia page: 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To scrape the Neighborhood data from the web page we will use the Beautiful Soup 4 package. We will use the 'lxml' parser to parse the HTML file.

Let us install Beautiful Soup and lxml

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 16.8MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 28.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


We will also need the 'requests' library to get the HTML data from the web page.

In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importing all necessary libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd

Let's get the source code from the Wikipedia page using the 'requests' library. 

In [5]:
# We use the 'text' method to get the html data in a string
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Now let's create a Beautiful Soup object

In [6]:
soup = BeautifulSoup(source, 'lxml')

# We can view the source code using print statement
# print(soup.prettify())

# I have commented this line as the source code can amount to hundreds and thousands of lines,
# and we all know that scrolling can be a real pain in the ass sometimes. You can always open
# the webpage and view source.

Now on inspecting the source code we can see that the table containing the neighborhood data is inside the html tag 'table' with class 'wikitable sortable'.

In [7]:
table = soup.find('table', class_='wikitable sortable')

#print(table.prettify())  # again commenting this as scrolling can be a pain in the ass.

Every row in the table is enclosed in the tag 'tr'. We will use this to fetch every row and append in a list.

In [8]:
toronto_raw = []
for row in table.find_all('tr'):
    neighborhood = row.text.split('\n')
    neighborhood[:] = (value for value in neighborhood if value != '')
    toronto_raw.append(neighborhood)

Now let us create a Pandas dataframe from this list and name it 'toronto'.

In [10]:
toronto = pd.DataFrame(toronto_raw[1:])
toronto_raw[0][0] = 'PostalCode'
toronto_raw[0][2] = 'Neighborhood'
toronto.columns = toronto_raw[0]
toronto

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


We will remove every row where the value of 'Borough' is 'Not assigned'.

In [11]:
toronto.drop(toronto.loc[toronto['Borough']=='Not assigned'].index, inplace=True)
toronto.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


For rows where 'Borough' has a proper value, but the 'Neighborhood' is 'Not assigned', we will set the neighborhood same as the Borough. For example, for Borough 'Queen's Park' we'll set the neighborhood also as 'Queen's Park'.

In [12]:
idx = toronto.loc[toronto['Neighborhood']=='Not assigned'].index  #fetching the index for the rows with neighborhood not assigned
toronto.loc[idx, ['Neighborhood']] = toronto['Borough'][idx]  # setting neighborhood same as borough for these rows

toronto.reset_index(drop=True, inplace=True)   #resetting index to consecutive numbers starting from zero
toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Many postal codes have multiple rows. We will group these rows into a single row and list all the Neighborhoods in a single cell separated by a comma. We are doing this as later we will cluster the neighborhoods based on the postal code, meaning, we will consider one postal code as one neighborhood.

In [13]:
toronto = toronto.groupby(['PostalCode','Borough'],as_index=False, sort=False).agg(", ".join)
toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


Let us see the number of rows in our dataframe.

In [14]:
print('The number of distinct neighborhoods in our data is ' + str(toronto.shape[0]) + '.')

The number of distinct neighborhoods in our data is 103.


### Part 2: Fetch the geographical coordinates for every neighborhood.

Now that we have the list of all neighborhoods in Toronto, lets fetch the geographical coordinates, i.e., the latitude and longitude for each of these. We will use Geocoder package for this.

Let us go ahead and install it.

In [15]:
pip install geocoder

     |████████████████████████████████| 102kB 19.8MB/s ta 0:00:01
     |████████████████████████████████| 829kB 37.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 21.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


Lets import the package.

In [16]:
import geocoder

We'll loop over the dataframe 'toronto' to fetch the coordinates one by one for each neighborhood. We'll also set a count to check if we successfully fetch the coordinates for every neighborhood.

Note - I was earlier using 'geocoder.google', however, it was not very reliable as it would return a null value many times. To prevent that I place a while loop which is now commented. This was basically done to ensure that every neighborhood's coordinates were fetched. Later I switched to 'geocoder.arcgis' and now we don't need the while loop anymore :).

In [17]:
coordinates = []
count = 0
for index, row in toronto.iterrows():
    lat_lng_coords = None

    # loop until you get the coordinates
    # while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(row[0]))
    lat_lng_coords = g.latlng
    if lat_lng_coords is not None:
        count = count + 1

    coordinates.append(lat_lng_coords)
    
print(count)        # print count to cross check that we have fetched coordinates for each neighborhood             
print(coordinates)  # lets look at the list of coordinates

103
[[43.752440000000036, -79.32927072599995], [43.730420577000075, -79.31331999999998], [43.65512000000007, -79.36263979699999], [43.72312500000004, -79.45158914699994], [43.66110229800006, -79.39103499999999], [43.662242201000026, -79.52837877199994], [43.811525000000074, -79.19551721399995], [43.74919500000004, -79.36190541699995], [43.707535000000064, -79.31177329699995], [43.65736301100003, -79.37817999999999], [43.70799000000005, -79.44836733199998], [43.64969222700006, -79.55394499999994], [43.78573000000006, -79.15874999999994], [43.72142500000007, -79.34345278999996], [43.68966500000005, -79.30716910999996], [43.65121000000005, -79.37548057699996], [43.692105179000066, -79.43035499999996], [43.648573449000025, -79.57824999999997], [43.76569000000006, -79.17525603599995], [43.67684518300007, -79.29522499999996], [43.64516015600003, -79.37367499999993], [43.68864000000008, -79.45101525299998], [43.76835912100006, -79.21758999999997], [43.70976500000006, -79.36390090899994], [43.

Lets now add these coordinates to the corresponding rows in our neighborhood dataframe.

In [18]:
toronto_data = pd.concat([toronto, pd.DataFrame(coordinates)], axis = 1)
toronto_data.rename(columns={0: 'Latitude', 1: 'Longitude'}, inplace = True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


### Part 3: Visualisation - Create a map of Toronto marking all the neighborhoods

Lets fetch the coordinates for Toronto City.

In [19]:
toronto_latlng = None
g = geocoder.arcgis('Toronto, Ontario')
toronto_latlng = g.latlng
print('The geograpical coordinates of Toronto are {}, {}.'.format(toronto_latlng[0], toronto_latlng[1]))

The geograpical coordinates of Toronto are 43.648690000000045, -79.38543999999996.


Import all the necessary libraries. In Part 3 we will only need the folium library to create a map. THe rest of the libraries will be required in later parts.

In [20]:
import numpy as np # library to handle data in a vectorized manner

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from scikit learn library to cluster the neighborhoods
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Now lets create the map. We'll add a circular marker to the coordinates for every neighborhood. We will also add popup labels to every marker showing the Neighborhood and Borough name.

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latlng[0], toronto_latlng[1]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],
                                           toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Part 4: Use the Foursquare API to explore the neighborhoods.

In [22]:
CLIENT_ID = 'D15BLSVCZY13S41MFADMLTK0LF4PG0U0KV51BWHOWISRGUJM' # Foursquare ID
CLIENT_SECRET = 'YF0PSDKAC02ZJIVDL3KTEQVD555SF2440WRM33GPZC0DQCXF' # Foursquare Secret
VERSION = '20190901' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

To start with, let us just explore the first neighborhood. Lets fetch the name and coordinates of this neighborhood from our dataframe.

In [23]:
toronto_data.loc[0, 'Neighborhood']

'Parkwoods'

In [24]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.752440000000036, -79.32927072599995.


Now to make a call to the Foursquare API we'll create a GET request URL. We will fetch the top hundred venues in the neighborhood within a radius of 500 metres.

Note - It is not necessary that every neighborhood will return 100 venues. It is just the upper limit.

In [25]:
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=D15BLSVCZY13S41MFADMLTK0LF4PG0U0KV51BWHOWISRGUJM&client_secret=YF0PSDKAC02ZJIVDL3KTEQVD555SF2440WRM33GPZC0DQCXF&ll=43.752440000000036,-79.32927072599995&v=20190901&radius=500&limit=100'

Now send the GET request and fetch the data in a JSON file.

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d83d6939ba3e50039a9a17e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.75694000450004,
    'lng': -79.32305255090476},
   'sw': {'lat': 43.74793999550003, 'lng': -79.33548890109513}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs

By examining the results we can see that all all the information is in the *items* key. Let's define a function that extracts the category of every venue.

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we will clean the json and structure it into a *pandas* dataframe.

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,KFC,Fast Food Restaurant,43.754387,-79.333021


Let's check how many venues were returned by Foursquare.

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's run the above function on every neighbourhood and create a new dataframe *toronto_venues* to put our results in

In [31]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

Let's print the size of the resulting dataframe.

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(2477, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752440,-79.329271,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752440,-79.329271,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.752440,-79.329271,KFC,43.754387,-79.333021,Fast Food Restaurant
3,Victoria Village,43.730421,-79.313320,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.730421,-79.313320,Memories of Africa,43.726602,-79.312427,Grocery Store


Further down in the notebook I found out that some venues were fetched with category as *Neighborhood*. We need to remove these venues as we will be using the venue categories to cluster the neighborhoods. Let's see which venues are these.

In [33]:
print(toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'])

                  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
427                The Beaches              43.676845              -79.295225   
549         Central Bay Street              43.656091              -79.384930   
734   Adelaide, King, Richmond              43.649515              -79.382503   
1285           Studio District              43.662766              -79.334830   

                 Venue  Venue Latitude  Venue Longitude Venue Category  
427      Upper Beaches       43.680563       -79.292869   Neighborhood  
549   Downtown Toronto       43.653232       -79.385296   Neighborhood  
734   Downtown Toronto       43.653232       -79.385296   Neighborhood  
1285       Leslieville       43.662070       -79.337856   Neighborhood  


Now let's drop these venues from the dataframe.

In [34]:
toronto_venues.drop(toronto_venues.loc[toronto_venues['Venue Category'] == 'Neighborhood'].index, inplace=True)

Let's check how many venues were returned for each neighborhood.

In [35]:
toronto_venues.groupby('Neighborhood').count().reset_index()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",99,99,99,99,99,99
1,Agincourt,15,15,15,15,15,15
2,"Agincourt North, L'Amoreaux East, Milliken, St...",3,3,3,3,3,3
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",13,13,13,13,13,13
4,"Alderwood, Long Branch",5,5,5,5,5,5
5,"Bathurst Manor, Downsview North, Wilson Heights",1,1,1,1,1,1
6,Bayview Village,4,4,4,4,4,4
7,"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
8,Berczy Park,62,62,62,62,62,62
9,"Birch Cliff, Cliffside West",6,6,6,6,6,6


Let's also check how many unique categories we have fetched.

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


### Part 5: Analyze each neighborhood and prepare the data for clustering.

Let's create a one-hot encoding for every venue category.

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's check the shape of the resulting dataframe.

In [38]:
toronto_onehot.shape

(2473, 261)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.00,0.030303,0.000000,0.010101,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.020202,0.000000,0.030303,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,0.

Let's confirm the size of the new dataframe. Note that it is possible that the number of neighborhoods might be less than in the original dataframe as no venues might have been fetched for some.

In [40]:
toronto_grouped.shape

(102, 261)

Let's print the top 5 venue category for each neighborhood

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2        Hotel  0.04
3   Restaurant  0.03
4          Bar  0.03


----Agincourt----
                  venue  freq
0         Shopping Mall  0.13
1    Chinese Restaurant  0.13
2                Bakery  0.07
3         Grocery Store  0.07
4  Hong Kong Restaurant  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0          Pharmacy  0.67
1  Sushi Restaurant  0.33
2               ATM  0.00
3            Museum  0.00
4     Moving Target  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.15
1   Japanese Restaurant  0.08
2  Fast Food Restaurant  0.08
3          Liquor Store  0.08
4            Beer Store  0.08


----Alderwood, Long Branch----
                venue  freq
0  Athletics & Sports   0.2
1   Convenience Store   0.2
2

Let's put this data into a dataframe. First let's define a function to to sort the venue categories in descending order of their frequency in a neighborhood.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now use the above function to fetch the top 10 venue categories for every neighborhood and put them in a dataframe.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Bar,Steakhouse,Burger Joint,Asian Restaurant,Gastropub
1,Agincourt,Chinese Restaurant,Shopping Mall,Hong Kong Restaurant,Department Store,Supermarket,Bakery,Discount Store,Pool,Grocery Store,Shanghai Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Pharmacy,Park,Hardware Store,Sandwich Place,Fast Food Restaurant,Japanese Restaurant,Liquor Store
4,"Alderwood, Long Branch",Pub,Athletics & Sports,Convenience Store,Sandwich Place,Gym,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market


### Part 6: Cluster the neighborhoods.

Finally it's time to cluster the neighborhood. Run *k-means* on the neighborhood data and let's divide them into 5 clusters.

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 4, 4, 1, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 3, 4, 3, 4, 4,
       4, 4, 3, 4, 3, 4, 4, 4, 3, 3, 4, 3, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 0, 3, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4,
       4, 3, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

Note - While joining this new dataframe with the old dataframe *toronto_data* it is important to make an *inner join*. This is important as the number of neighbourhoods might not be same in both the neighbourhoods. This will result in the non common neighborhoods to have the value *NaN* as a cluster label.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752440,-79.329271,3,Food & Drink Shop,Park,Fast Food Restaurant,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
1,M4A,North York,Victoria Village,43.730421,-79.313320,3,French Restaurant,Grocery Store,Park,Intersection,Yoga Studio,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640,4,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Yoga Studio,Theater,Pub,Restaurant,Electronics Store,Event Space,Mexican Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589,4,Clothing Store,Dessert Shop,Men's Store,Cosmetics Shop,American Restaurant,Electronics Store,Food Court,Furniture / Home Store,Toy / Game Store,Fast Food Restaurant
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035,4,Coffee Shop,Sandwich Place,Café,Indian Restaurant,Bubble Tea Shop,Italian Restaurant,Art Gallery,College Auditorium,Burrito Place,Smoothie Shop


Finally, let's visualize the resulting clusters

In [46]:
# create map
map_clusters = folium.Map(location=[toronto_latlng[0], toronto_latlng[1]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters